# SQL in Python Assignment

In [1]:
import pandas as pd
from sqlalchemy import *

### Connect to the personal database you created yesterday.
Remember the way the credentials work:
```
user = 'postgres' # Replace with your user, if different
pwd = 'Thinkful' # Replace with your password
host = 'localhost'
port = '5432'
db = 'postgres' # Replace with your database name, if different
```

In [29]:
user = 'postgres' # Replace with your user, if different
pwd = 'Thinkful' # Replace with your password
host = 'localhost'
port = '5432'
db = 'postgres' # Replace with your database name, if different

sql = "SELECT * FROM vehicles"

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    user, pwd, host, port, db))

# Since empno is a primary key for each employee, it works great as an index
df = pd.read_sql_query(sql,con=engine)

# no need for an open connection, as we're only doing a single query
#engine.dispose()

df.head(10)

,id,make,model,year,class,trans,drive,cyl,displ,fuel,hwy,cty
0,27550,AM General,DJ Po Vehicle 2WD,1984,Special Purpose Vehicle 2WD,Automatic 3-spd,2-Wheel Drive,4,2.5,Regular,17,18
1,28426,AM General,DJ Po Vehicle 2WD,1984,Special Purpose Vehicle 2WD,Automatic 3-spd,2-Wheel Drive,4,2.5,Regular,17,18
2,27549,AM General,FJ8c Post Office,1984,Special Purpose Vehicle 2WD,Automatic 3-spd,2-Wheel Drive,6,4.2,Regular,13,13
3,28425,AM General,FJ8c Post Office,1984,Special Purpose Vehicle 2WD,Automatic 3-spd,2-Wheel Drive,6,4.2,Regular,13,13
4,1032,AM General,Post Office DJ5 2WD,1985,Special Purpose Vehicle 2WD,Automatic 3-spd,Rear-Wheel Drive,4,2.5,Regular,17,16
5,1033,AM General,Post Office DJ8 2WD,1985,Special Purpose Vehicle 2WD,Automatic 3-spd,Rear-Wheel Drive,6,4.2,Regular,13,13
6,3347,ASC Incorporated,GNX,1987,Midsize Cars,Automatic 4-spd,Rear-Wheel Drive,6,3.8,Premium,21,14
7,13309,Acura,2.2CL/3.0CL,1997,Subcompact Cars,Automatic 4-spd,Front-Wheel Drive,4,2.2,Regular,26,20
8,13310,Acura,2.2CL/3.0CL,1997,Subcompact Cars,Manual 5-spd,Front-Wheel Drive,4,2.2,Regular,28,22
9,13311,Acura,2.2CL/3.0CL,1997,Subcompact Cars,Automatic 4-spd,Front-Wheel Drive,6,3,Regular,26,18


### Obtain a list of tables that are in the database.

In [17]:
engine.table_names()

['vehicles', 'houseprices', 'normal_sales']

### Obtain a list of all the fields that are in the vehicles table along with the data type of each.

In [26]:

meta = MetaData()
table = Table('vehicles', meta, autoload=True, autoload_with=engine)
fields = [(c.name, c.type) for c in table.columns]
fields_df = pd.DataFrame(fields, columns=['Name', 'Type'])
fields_df

,Name,Type
0,id,INTEGER
1,make,VARCHAR(50)
2,model,VARCHAR(50)
3,year,VARCHAR(50)
4,class,VARCHAR(255)
5,trans,VARCHAR(50)
6,drive,VARCHAR(50)
7,cyl,VARCHAR(50)
8,displ,VARCHAR(50)
9,fuel,VARCHAR(50)


In [10]:
df.dtypes

id        int64
make     object
model    object
year     object
class    object
trans    object
drive    object
cyl      object
displ    object
fuel     object
hwy      object
cty      object
dtype: object

### Write a SELECT query to retrieve all records from the database and load them into a Pandas data frame.

In [11]:
#SEE LINE BLOCK FOUR FOR SQL SELECT QUERY AND CONNECTION
df.head()

,id,make,model,year,class,trans,drive,cyl,displ,fuel,hwy,cty
0,27550,AM General,DJ Po Vehicle 2WD,1984,Special Purpose Vehicle 2WD,Automatic 3-spd,2-Wheel Drive,4,2.5,Regular,17,18
1,28426,AM General,DJ Po Vehicle 2WD,1984,Special Purpose Vehicle 2WD,Automatic 3-spd,2-Wheel Drive,4,2.5,Regular,17,18
2,27549,AM General,FJ8c Post Office,1984,Special Purpose Vehicle 2WD,Automatic 3-spd,2-Wheel Drive,6,4.2,Regular,13,13
3,28425,AM General,FJ8c Post Office,1984,Special Purpose Vehicle 2WD,Automatic 3-spd,2-Wheel Drive,6,4.2,Regular,13,13
4,1032,AM General,Post Office DJ5 2WD,1985,Special Purpose Vehicle 2WD,Automatic 3-spd,Rear-Wheel Drive,4,2.5,Regular,17,16


### Create a table that contains all Audi vehicles with a city fuel efficiency greater than 18.

In [31]:
create = """
CREATE TABLE IF NOT EXISTS audi AS
SELECT * 
FROM vehicles 
WHERE make = 'Audi'and cty::int > 18
"""
#print(create)
engine.execute(create)

### Delete all records from the temp table you just created.

In [34]:
delete = """
delete FROM  audi
"""
engine.execute(delete)

### Insert records from the vehicles table back into the table you created where the make is Audi and the city fuel efficiency is greater than 20.

In [37]:
insert = """
INSERT INTO audi 
SELECT * 
FROM vehicles 
WHERE make = 'Audi'and cty::int > 20
"""
engine.execute(insert)

### Update the drive field of the records in the table you created to change any "4-Wheel or All-Wheel Drive" values to "All-Wheel Drive."

In [38]:
update = """
UPDATE audi 
SET drive = 'All-Wheel Drive'
WHERE drive = '4-Wheel or All-Wheel Drive'
"""
engine.execute(update)

### Write a for loop that iterates over the list of makes provided below, selects the top 10 vehicles with the highest city fuel efficiency for each make, and then concatenates all the results together into a single Pandas data frame.

In [40]:
makes = ['Ford', 'Chevrolet', 'Toyota', 'Honda']

In [44]:
results = []

for make in makes:
    sql = "SELECT * FROM vehicles WHERE make = '" + make + "' ORDER BY cty DESC LIMIT 10"
    
    result = engine.execute(sql)
    results.append(pd.DataFrame(result, columns=list(fields_df.Name)))
results_df = pd.concat(results)

In [45]:
results_df.head()

,id,make,model,year,class,trans,drive,cyl,displ,fuel,hwy,cty
0,1586,Ford,E250 Econoline 2WD,1985,Vans,Automatic 3-spd,Rear-Wheel Drive,8,5.8,Regular,10,9
1,28416,Ford,E150 Club Wagon 2WD,1984,"Vans, Passenger Type",Automatic 3-spd,2-Wheel Drive,8,5.8,Regular,10,9
2,28394,Ford,E250 Econoline 2WD,1984,"Vans, Cargo Type",Automatic 3-spd,2-Wheel Drive,8,5.8,Regular,10,9
3,30966,Ford,Ranger Pickup 2WD,2000,Standard Pickup Trucks 2WD,Automatic (A1),2-Wheel Drive,NULL,NULL,Electricity,54,62
4,30965,Ford,Ranger Pickup 2WD,2001,Standard Pickup Trucks 2WD,Automatic (A1),2-Wheel Drive,NULL,NULL,Electricity,54,62
